**Column Details:**
* **Unnamed 0:** It has the id number of the news.
* **title :** Contains the title of the article or news.
* **text :** This column has the actual written news
* **class :** Target class representing FAKE OR REAL news

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

tf.config.run_functions_eagerly(True)

In [2]:
df = pd.read_csv("/content/news_2.csv")
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
df.shape

(6335, 4)

In [4]:
df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [5]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
df['label'].value_counts()

,count
label,
REAL,3171
FAKE,3164


In [7]:
d = {"FAKE":0, "REAL":1}
df['label'] = df['label'].map(d)

In [8]:
# Let's initialize some variables for model training

embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = 0.1

In [9]:
title, text, labels = [], [], []
for x in range(training_size):
    title.append(df['title'][x])
    text.append(df['text'][x])
    labels.append(df['label'][x])

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(title)
word_index = tokenizer.word_index
vocab_size = len(word_index)
sequences = tokenizer.texts_to_sequences(title)

padded = pad_sequences(
    sequences=sequences,
    padding=padding_type,
    truncating = trunc_type
    )

split = int(test_portion*training_size)
training_sequences = padded[split:training_size]
test_sequences = padded[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

embeddings_index = {}
with open('glove.6B.50d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

--2025-02-20 20:54:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-02-20 20:54:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-02-20 20:54:02--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

In [12]:
# Model Building
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size + 1, embedding_dim,
                       input_length = max_length,
                       weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │         377,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 377,600 (1.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 377,600 (1.44 MB)

In [14]:
num_epochs = 50
training_padded = np.array(training_sequences)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences)
testing_labels = np.array(test_labels)

train_ds = tf.data.Dataset.from_tensor_slices((training_padded, training_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((testing_padded, testing_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(train_ds, validation_data=test_ds, epochs=num_epochs, verbose=2)


Epoch 1/50
85/85 - 15s - 182ms/step - accuracy: 0.6089 - loss: 0.6445 - val_accuracy: 0.6967 - val_loss: 0.5565
Epoch 2/50
85/85 - 15s - 172ms/step - accuracy: 0.6989 - loss: 0.5742 - val_accuracy: 0.7200 - val_loss: 0.5297
Epoch 3/50
85/85 - 14s - 170ms/step - accuracy: 0.7307 - loss: 0.5292 - val_accuracy: 0.7600 - val_loss: 0.4941
Epoch 4/50
85/85 - 15s - 172ms/step - accuracy: 0.7759 - loss: 0.4697 - val_accuracy: 0.7767 - val_loss: 0.4693
Epoch 5/50
85/85 - 21s - 243ms/step - accuracy: 0.8041 - loss: 0.4182 - val_accuracy: 0.7600 - val_loss: 0.4735
Epoch 6/50
85/85 - 15s - 175ms/step - accuracy: 0.8252 - loss: 0.3895 - val_accuracy: 0.7433 - val_loss: 0.5563
Epoch 7/50
85/85 - 16s - 187ms/step - accuracy: 0.8393 - loss: 0.3610 - val_accuracy: 0.7600 - val_loss: 0.5888
Epoch 8/50
85/85 - 20s - 234ms/step - accuracy: 0.8604 - loss: 0.3138 - val_accuracy: 0.7800 - val_loss: 0.5614
Epoch 9/50
85/85 - 15s - 172ms/step - accuracy: 0.8804 - loss: 0.2858 - val_accuracy: 0.7633 - val_loss:

In [ ]:
x = "What's in that Iran bill that Obama doesn't like?"                   #Real news example from dataset
# x = "3 Effects of Substance Abuse on Individual, Family and Community"  #Fake news example from dataset

sequences = tokenizer.texts_to_sequences([x])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding = padding_type,
                          truncating = trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is REAL")
else:
    print("This news is FAKE")

Thank you for going throgh this work till the end, feel free to get in touch if you have any questions... :)